<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->




# **P1 - RA ÍMPAR**

In [184]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Eduardo Lucas Felippa, 20019130' #@param {type:"string"}






In [185]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

path = 'https://github.com/Rogerio-mack/work/raw/main/data/'

# **Case**: Airbnb

Neste case, analise dados sobre imóveis alugados pela plataforma Airbnb em NYC no ano de 2019.


In [186]:
df = pd.read_csv(path + 'AB_NYC_2019.csv')
df.head()

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.57                               1                 0  
1               1.46                               2               137  
2               3.97                               3               182  
3               0.91                               2               286  
4               0.43                               2                59

# Q1. Proprietários com mais imóveis na plataforma

Obtenha os 3 proprietários ("hosts") com mais imóveis sendo anunciados na plataforma, e a respectiva quantidade de imóveis anunciados.


In [233]:
hosts = df.value_counts('host_id').sort_values(ascending=False)
hosts[0:3]
df[df.host_id.isin(hosts[0:3].index)][["host_id", "host_name"]].value_counts()

host_id    host_name   
219517861  Sonder (NYC)    7
30283594   Kara            4
120762452  Stanley         4
Name: count, dtype: int64

# Q2B. Preço nos bairros com mais imóveis

 Qual o preço médio dos imóveis, considerando somente os preços dos imóveis cujos bairros (`neighbourhood`) tenham mais de 20 imóveis sendo anunciados?  


In [188]:
neighbourhood = df.value_counts('neighbourhood').sort_values(ascending=False)
neighbourhood = neighbourhood[neighbourhood > 20]
preco_medio = df[df.neighbourhood.isin(neighbourhood.index)].price.mean()

print(f'O preço médio dos imóveis em bairros com mais de 20 anúncios é: USD {preco_medio:.2f}')

O preço médio dos imóveis em bairros com mais de 20 anúncios é: USD 151.42


# Q3B.  Brooklyn *outliers*

Qual o percentual de imóveis de Brooklyn com preços estatisticamente discrepantes (*outliers*) com relação aos preços de todos os imóveis?



In [189]:
# outliers são dados fora do intervalo  [Q1−1.5IQR,Q3+1.5IQR], sendo IQR = Q3 - Q1

df_Brooklyn = df[df['neighbourhood_group']=='Brooklyn']
df_Brooklyn.describe()
Q1 = df.price.quantile(0.25)
Q3 = df.price.quantile(0.75)
IQR = Q3 - Q1
outliers = df_Brooklyn[(df_Brooklyn.price < (Q1 - 1.5 * IQR)) | (df_Brooklyn.price > (Q3 + 1.5 * IQR))]
perc_Brooklyn = (len(outliers)/len(df))*100

print( f'Percentuais de outliers de preços, Brooklyn: {perc_Brooklyn:.2f} %')


Percentuais de outliers de preços, Brooklyn: 1.62 %


# Q4B. Scores

Nesta tarefa você deve atribuir a cada imóvel um valor de *score* (`df['score']`) baseado nas seguintes regras:

1. Imóveis com `availability_365` > 90 recebem +10 pontos enquanto imóveis com `availability_365` < 5 recebem -5 pontos;
2. Imóveis com o número de reviews >= 10 recebem +5 pontos para cada 10 reviews (por ex. se número de reviews = 20, são atribuídos +10 pontos). Se número de reviews = 0 o imóvel recebe -5 pontos;
3. Imóveis com preço abaixo da média de seu `neighbourhood`	recebem +5 pontos.

Implemente empregando `apply()` ou um laço de programa, como preferir.

In [183]:
media_neighbourhood = df.groupby('neighbourhood').price.mean()
df['score'] = 0
df.loc[df.availability_365 > 90, 'score'] += 10
df.loc[df.availability_365 < 5, 'score'] -= 5
df.loc[df.number_of_reviews >= 10, 'score'] += (df.number_of_reviews // 10) * 5
df.loc[df.number_of_reviews == 0, 'score'] -= 5
# df.loc[df.price < media_neighbourhood[df.neighbourhood], 'score'] += 5


def score(row):
  if row.price < media_neighbourhood[row.neighbourhood]:
    return row.score + 5
  else:
    return row.score

df.loc['score'] = df.apply(score, axis=1)

# imovel = df[df.id == 20913]

# for imovel in df.id:
#   imovel = df[df.id == imovel]
#   if imovel.price < media_neighbourhood[imovel.neighbourhood[0]]:
#     df.loc[df.id == imovel.id, 'score'] += 5


# df.head()
# df.loc[df.price < media_neighbourhood[ ]]

print(f'A soma dos valores de score é: ', df['score'].sum())


A soma dos valores de score é:  8550.0


# Q5. Enriquecendo os dados

Enriqueça os dados dos imóveis com os valores dos impostos e multas municipais. Cacule a média de impostos e multas de cada grupo de vizinhaça, `neighbourhood_group`. Note que existem imóveis sem impostos e sem multas, esses são imóveis isentos, mas devem ser considerados nos cálculos de média (por exemplo, você pode considerar esses imóveis com valor 0).

In [228]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_taxes.loc[df_taxes['value'].isna(), 'value'] = 0
manhattan_idxs = df[df.neighbourhood_group == 'Manhattan'].id.array
mean_tax = df_taxes[(df_taxes.id.isin(manhattan_idxs)) & (df_taxes.type == 'taxes')]['value'].mean()
mean_fine = df_taxes[(df_taxes.id.isin(manhattan_idxs)) & (df_taxes.type == 'fines')]['value'].mean()
print(f'A média de impostos de Manhattan é: {int(mean_tax)}')
print(f'A média de multas de Manhattan é: {int(mean_fine)}')

A média de impostos de Manhattan é: 888
A média de multas de Manhattan é: 314


# Q6. Quebras

Construa no `Pandas` um "relatório" (`df_report`) como abaixo. Os valores são ordenados `neighbourhood_group`,`neighbourhood`. Cada "página" contém apenas 30 linhas. A cada "quebra" de `neighbourhood_group` é gerada uma linha com preço médio do `neighbourhood_group` e, então, é iniciada uma nova página.

O item `line_id` é opcional mas deve auxiliá-lo nessa tarefa.

Ao final produza a soma dos preços das páginas abaixo...

In [229]:


#sum_price_pages = df_report[ df_report.page_id.isin([1,24,25,26,28]) ].groupby('page_id').price.sum().astype(int).sum()

#print(f'A soma dos preços das páginas 1, 24, 25 e 28 é: {sum_price_pages}')

NameError: name 'df_report' is not defined

In [232]:
# Create a copy of the DataFrame to avoid modifying the original
df_report = df.copy()

# Group data by 'neighbourhood_group' and 'neighbourhood'
df_report = df_report.groupby(['neighbourhood_group', 'neighbourhood']).agg({'price': 'mean'})

# Sort by 'neighbourhood_group' and 'neighbourhood'
df_report = df_report.sort_values(['neighbourhood_group', 'neighbourhood'])

# Reset index for easier manipulation
df_report = df_report.reset_index()

# Initialize variables for page and line ID
page_id = 1
line_id = 1

# Initialize an empty list to store the report data
report_data = []

# Iterate through each group
for group_name, group_data in df_report.groupby('neighbourhood_group'):
  # Calculate the average price for the neighbourhood group
  avg_price_group = group_data['price'].mean()

  # Add a row for the neighbourhood group average price
  report_data.append({
      'neighbourhood_group': group_name,
      'neighbourhood': 'Average Price for Neighbourhood Group',
      'price': avg_price_group,
      'page_id': page_id,
      'line_id': line_id
  })
  line_id += 1

  # Split into pages
  for i in range(0, len(group_data), 30):
    page_data = group_data[i:i+30]
    for index, row in page_data.iterrows():
      report_data.append({
          'neighbourhood_group': row['neighbourhood_group'],
          'neighbourhood': row['neighbourhood'],
          'price': row['price'],
          'page_id': page_id,
          'line_id': line_id
      })
      line_id += 1

    page_id += 1
    line_id = 1  # Reset line_id for a new page

# Create the DataFrame
df_report = pd.DataFrame(report_data)

# Calculate and print the sum of prices for the specified pages
sum_price_pages = df_report[
    df_report.page_id.isin([1, 24, 25, 26, 28])
].groupby('page_id').price.sum().astype(int).sum()

print(f'A soma dos preços das páginas 1, 24, 25, 26 e 28 é: {sum_price_pages}')


A soma dos preços das páginas 1, 24, 25, 26 e 28 é: 1099


#### Aqui um *sample* do report.

In [ ]:
df_report.iloc[-10::,:]

neighbourhood_group  neighbourhood       price  page_id  line_id
795              Queens       Woodside   75.000000       28       17
796              Queens       Woodside  120.000000       28       18
797              Queens           MEAN   92.227848       28       19
798       Staten Island  Randall Manor   99.000000       29        1
799       Staten Island    South Beach   20.000000       29        2
800       Staten Island    South Beach  100.000000       29        3
801       Staten Island     St. George  130.000000       29        4
802       Staten Island      Todt Hill  135.000000       29        5
803       Staten Island  Tompkinsville   59.000000       29        6
804       Staten Island           MEAN   89.428571       29        7

In [ ]:
# prompt: Gere um texto explicando o que é e quais sao as fases do CRISP DM

O CRISP-DM (Cross-Industry Standard Process for Data Mining) é um modelo de processo de mineração de dados que fornece um guia estruturado para projetos de análise de dados. Ele descreve um ciclo de vida que inclui seis fases principais, interligadas e iterativas:

**1. Entendimento do Negócio:**  Nesta fase inicial, o foco é compreender completamente o problema de negócio que se pretende resolver com a mineração de dados.  Isso envolve identificar as metas de negócio, definir o escopo do projeto, identificar as necessidades dos stakeholders e definir as medidas de sucesso.  É crucial estabelecer uma clara definição do problema e dos objetivos a serem alcançados.

**2. Entendimento dos Dados:** Uma vez definido o problema de negócio, a próxima etapa é coletar e analisar os dados relevantes. Isso inclui identificar as fontes de dados, avaliar a qualidade dos dados, explorando os dados para identificar tendências, padrões e anomalias, e entender a relação entre os dados e o problema de negócio.  É crucial determinar se os dados disponíveis são suficientes para atingir os objetivos definidos na primeira fase e identificar possíveis problemas de qualidade dos dados, como valores faltantes ou inconsistentes.

**3. Preparação dos Dados:** Esta fase é crucial e frequentemente a mais demorada no processo.  Ela envolve a limpeza dos dados, a transformação dos dados para um formato adequado para a modelagem, e a seleção das variáveis relevantes para o modelo.  Aqui, técnicas como tratamento de valores ausentes, lidar com outliers, transformação de variáveis e redução de dimensionalidade são comumente aplicadas.  O objetivo é obter um conjunto de dados limpo, consistente e pronto para ser utilizado no processo de modelagem.

**4. Modelagem:** Nesta fase, os modelos de mineração de dados são selecionados e aplicados aos dados preparados.  Vários algoritmos de aprendizado de máquina podem ser usados, dependendo do problema de negócio e da natureza dos dados. O processo inclui treinamento e avaliação dos modelos, seleção dos melhores modelos e ajuste dos parâmetros para otimizar o desempenho. A iteração entre esta fase e a de preparação de dados é comum, pois a qualidade dos dados afeta diretamente o desempenho dos modelos.

**5. Avaliação:** Uma vez que os modelos tenham sido treinados e ajustados, a fase de avaliação foca em avaliar o seu desempenho e utilidade em relação aos objetivos de negócio definidos inicialmente. Métricas como precisão, revocação, F1-score, AUC e outras, relevantes para o tipo de problema, são utilizadas para avaliar o desempenho preditivo dos modelos.  A avaliação pode também levar à revisão das fases anteriores, como a preparação de dados ou a seleção do modelo.  O objetivo é garantir que o modelo escolhido seja o mais apropriado para atender às necessidades do negócio.

**6. Implantação:**  Esta é a fase final do processo, na qual o modelo é colocado em produção e usado para tomada de decisão. Isso pode envolver a integração do modelo em sistemas existentes, a criação de um sistema de monitoramento do modelo, e a documentação do processo completo.  A manutenção e atualização do modelo ao longo do tempo são também parte crucial desta fase para assegurar que o modelo continue a gerar resultados relevantes e precisos.  A implantação garante que os resultados da mineração de dados sejam transformados em valor real para o negócio.

Vale ressaltar que o CRISP-DM é um processo iterativo. Isso significa que, em muitas ocasiões, será necessário voltar a etapas anteriores para refiná-las e obter melhores resultados.
